# Data Preprocessing

In [12]:
# importing necessary packages
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import linear_model
import matplotlib.pyplot as plt

In [13]:
def coef_graph():
    # plotting columns and their coef impact on predictions
    coefs = pd.DataFrame(
    regr.coef_,
    #  columns=['Coefficients'], index=X_train.columns
    columns=['Coefficients'], index=X.columns
    )

    # sort coefficients highest to lowest
    coefs.sort_values(by=['Coefficients'], inplace=True)

    # function to add value labels
    def addlabels(x,y):
        for i in range(len(x)):
            plt.text(i,y[i],y[i])

    # creating the graph
    coefs.plot(kind='barh', figsize=(9, 7))
    plt.title('Linear Regression Model')
    plt.axvline(x=0, color='.5')
    plt.subplots_adjust(left=.3)

In [14]:
# loading in data cleaned in data-preparation.ipynb
svi_mortality_2014 = pd.read_csv('./data/svi-mortality-county/2014-svi-mortality.csv', index_col=0)
svi_mortality_2020 = pd.read_csv('./data/svi-mortality-county/2020-svi-mortality.csv', index_col=0)

# rename columns to match
svi_mortality_2020.rename(columns={'E_POV150': 'E_POV'}, inplace=True)

# drop columns with no equivalent
svi_mortality_2014.drop('E_PCI', axis=1, inplace=True)
svi_mortality_2020.drop(['E_HBURD', 'E_NOINT', 'E_AFAM', 'E_HISP', 'E_ASIAN', 'E_AIAN', 'E_NHPI', 'E_TWOMORE', 'E_OTHERRACE'], axis=1, inplace=True)

# create new df with calculated change in estimated values
diff_2020_2014 = svi_mortality_2020 - svi_mortality_2014

# drop all na rows
diff_2020_2014.dropna(inplace=True)

In [15]:
svi_mortality_2014.head()

,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_NOHSDP,E_AGE65,E_AGE17,E_DISABL,E_SNGPNT,E_MINRTY,E_LIMENG,E_MUNIT,E_MOBILE,E_CROWD,E_NOVEH,E_GROUPQ,E_UNINSUR,E_DAYPOP,Deaths
1001,55136.0,22431.0,20304.0,7006.0,2252.0,5012.0,7321.0,14214.0,8700.0,1649.0,13103.0,249.0,842.0,4401.0,530.0,1081.0,442.0,5814.0,43534.0,114.0
1003,191205.0,105563.0,73058.0,25988.0,7856.0,14615.0,33782.0,43186.0,26603.0,5027.0,32158.0,2571.0,18988.0,12200.0,998.0,2242.0,2611.0,27758.0,177010.0,465.0
1005,27119.0,11833.0,9145.0,5832.0,1527.0,4790.0,4180.0,5862.0,4856.0,1098.0,14614.0,549.0,129.0,3175.0,178.0,802.0,2869.0,3760.0,29769.0,90.0
1007,22653.0,8985.0,7078.0,3596.0,975.0,3466.0,3209.0,4903.0,3343.0,637.0,5717.0,112.0,163.0,2379.0,17.0,299.0,1576.0,2725.0,19274.0,57.0
1009,57645.0,23868.0,20934.0,9866.0,2291.0,8567.0,9172.0,13877.0,9729.0,1650.0,6825.0,954.0,215.0,5497.0,344.0,823.0,572.0,6922.0,41857.0,143.0


In [16]:
diff_2020_2014.head()

,Deaths,E_AGE17,E_AGE65,E_CROWD,E_DAYPOP,E_DISABL,E_GROUPQ,E_HH,E_HU,E_LIMENG,E_MINRTY,E_MOBILE,E_MUNIT,E_NOHSDP,E_NOVEH,E_POV,E_SNGPNT,E_TOTPOP,E_UNEMP,E_UNINSUR
1001,41.0,-1071.0,1169.0,-191.0,-1724.0,958.0,136.0,1255.0,1266.0,114.0,1376.0,-88.0,76.0,-739.0,86.0,5605.0,-41.0,503.0,-1516.0,-1469.0
1003,119.0,3807.0,10934.0,282.0,41597.0,4012.0,343.0,10989.0,11184.0,-978.0,5176.0,-307.0,525.0,208.0,385.0,10425.0,-1710.0,27084.0,-3829.0,-7257.0
1005,16.0,-640.0,597.0,172.0,-2636.0,-697.0,41.0,177.0,224.0,-116.0,-920.0,469.0,41.0,-293.0,237.0,3133.0,-69.0,-2093.0,-878.0,-1398.0
1007,21.0,-319.0,467.0,74.0,-475.0,405.0,81.0,181.0,252.0,-37.0,7.0,564.0,65.0,-410.0,182.0,2134.0,153.0,-279.0,-308.0,-847.0
1009,55.0,-505.0,1210.0,41.0,315.0,-1165.0,-8.0,271.0,536.0,-153.0,865.0,546.0,-48.0,-1729.0,254.0,3758.0,-337.0,110.0,-1038.0,-1176.0


In [17]:
pd.set_option('display.max_columns', None)

In [18]:
diff_2020_2014 = diff_2020_2014.add_prefix('diff_')
ml_df = pd.concat([svi_mortality_2014, diff_2020_2014], axis=1)

In [19]:
ml_df.head()

,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_NOHSDP,E_AGE65,E_AGE17,E_DISABL,E_SNGPNT,E_MINRTY,E_LIMENG,E_MUNIT,E_MOBILE,E_CROWD,E_NOVEH,E_GROUPQ,E_UNINSUR,E_DAYPOP,Deaths,diff_Deaths,diff_E_AGE17,diff_E_AGE65,diff_E_CROWD,diff_E_DAYPOP,diff_E_DISABL,diff_E_GROUPQ,diff_E_HH,diff_E_HU,diff_E_LIMENG,diff_E_MINRTY,diff_E_MOBILE,diff_E_MUNIT,diff_E_NOHSDP,diff_E_NOVEH,diff_E_POV,diff_E_SNGPNT,diff_E_TOTPOP,diff_E_UNEMP,diff_E_UNINSUR
1001,55136.0,22431.0,20304.0,7006.0,2252.0,5012.0,7321.0,14214.0,8700.0,1649.0,13103.0,249.0,842.0,4401.0,530.0,1081.0,442.0,5814.0,43534.0,114.0,41.0,-1071.0,1169.0,-191.0,-1724.0,958.0,136.0,1255.0,1266.0,114.0,1376.0,-88.0,76.0,-739.0,86.0,5605.0,-41.0,503.0,-1516.0,-1469.0
1003,191205.0,105563.0,73058.0,25988.0,7856.0,14615.0,33782.0,43186.0,26603.0,5027.0,32158.0,2571.0,18988.0,12200.0,998.0,2242.0,2611.0,27758.0,177010.0,465.0,119.0,3807.0,10934.0,282.0,41597.0,4012.0,343.0,10989.0,11184.0,-978.0,5176.0,-307.0,525.0,208.0,385.0,10425.0,-1710.0,27084.0,-3829.0,-7257.0
1005,27119.0,11833.0,9145.0,5832.0,1527.0,4790.0,4180.0,5862.0,4856.0,1098.0,14614.0,549.0,129.0,3175.0,178.0,802.0,2869.0,3760.0,29769.0,90.0,16.0,-640.0,597.0,172.0,-2636.0,-697.0,41.0,177.0,224.0,-116.0,-920.0,469.0,41.0,-293.0,237.0,3133.0,-69.0,-2093.0,-878.0,-1398.0
1007,22653.0,8985.0,7078.0,3596.0,975.0,3466.0,3209.0,4903.0,3343.0,637.0,5717.0,112.0,163.0,2379.0,17.0,299.0,1576.0,2725.0,19274.0,57.0,21.0,-319.0,467.0,74.0,-475.0,405.0,81.0,181.0,252.0,-37.0,7.0,564.0,65.0,-410.0,182.0,2134.0,153.0,-279.0,-308.0,-847.0
1009,57645.0,23868.0,20934.0,9866.0,2291.0,8567.0,9172.0,13877.0,9729.0,1650.0,6825.0,954.0,215.0,5497.0,344.0,823.0,572.0,6922.0,41857.0,143.0,55.0,-505.0,1210.0,41.0,315.0,-1165.0,-8.0,271.0,536.0,-153.0,865.0,546.0,-48.0,-1729.0,254.0,3758.0,-337.0,110.0,-1038.0,-1176.0


In [20]:
ml_df.dropna(inplace=True)
y = ml_df.diff_Deaths
ml_df.drop('diff_Deaths', axis=1, inplace=True)

In [33]:
y

1001      41.0
1003     119.0
1005      16.0
1007      21.0
1009      55.0
         ...  
56037     33.0
56039      4.0
56041     11.0
56043      6.0
56045      4.0
Name: diff_Deaths, Length: 2847, dtype: float64

In [22]:
ml_df.columns

Index(['E_TOTPOP', 'E_HU', 'E_HH', 'E_POV', 'E_UNEMP', 'E_NOHSDP', 'E_AGE65',
       'E_AGE17', 'E_DISABL', 'E_SNGPNT', 'E_MINRTY', 'E_LIMENG', 'E_MUNIT',
       'E_MOBILE', 'E_CROWD', 'E_NOVEH', 'E_GROUPQ', 'E_UNINSUR', 'E_DAYPOP',
       'Deaths', 'diff_E_AGE17', 'diff_E_AGE65', 'diff_E_CROWD',
       'diff_E_DAYPOP', 'diff_E_DISABL', 'diff_E_GROUPQ', 'diff_E_HH',
       'diff_E_HU', 'diff_E_LIMENG', 'diff_E_MINRTY', 'diff_E_MOBILE',
       'diff_E_MUNIT', 'diff_E_NOHSDP', 'diff_E_NOVEH', 'diff_E_POV',
       'diff_E_SNGPNT', 'diff_E_TOTPOP', 'diff_E_UNEMP', 'diff_E_UNINSUR'],
      dtype='object')

In [23]:
ml_df.head()

,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_NOHSDP,E_AGE65,E_AGE17,E_DISABL,E_SNGPNT,E_MINRTY,E_LIMENG,E_MUNIT,E_MOBILE,E_CROWD,E_NOVEH,E_GROUPQ,E_UNINSUR,E_DAYPOP,Deaths,diff_E_AGE17,diff_E_AGE65,diff_E_CROWD,diff_E_DAYPOP,diff_E_DISABL,diff_E_GROUPQ,diff_E_HH,diff_E_HU,diff_E_LIMENG,diff_E_MINRTY,diff_E_MOBILE,diff_E_MUNIT,diff_E_NOHSDP,diff_E_NOVEH,diff_E_POV,diff_E_SNGPNT,diff_E_TOTPOP,diff_E_UNEMP,diff_E_UNINSUR
1001,55136.0,22431.0,20304.0,7006.0,2252.0,5012.0,7321.0,14214.0,8700.0,1649.0,13103.0,249.0,842.0,4401.0,530.0,1081.0,442.0,5814.0,43534.0,114.0,-1071.0,1169.0,-191.0,-1724.0,958.0,136.0,1255.0,1266.0,114.0,1376.0,-88.0,76.0,-739.0,86.0,5605.0,-41.0,503.0,-1516.0,-1469.0
1003,191205.0,105563.0,73058.0,25988.0,7856.0,14615.0,33782.0,43186.0,26603.0,5027.0,32158.0,2571.0,18988.0,12200.0,998.0,2242.0,2611.0,27758.0,177010.0,465.0,3807.0,10934.0,282.0,41597.0,4012.0,343.0,10989.0,11184.0,-978.0,5176.0,-307.0,525.0,208.0,385.0,10425.0,-1710.0,27084.0,-3829.0,-7257.0
1005,27119.0,11833.0,9145.0,5832.0,1527.0,4790.0,4180.0,5862.0,4856.0,1098.0,14614.0,549.0,129.0,3175.0,178.0,802.0,2869.0,3760.0,29769.0,90.0,-640.0,597.0,172.0,-2636.0,-697.0,41.0,177.0,224.0,-116.0,-920.0,469.0,41.0,-293.0,237.0,3133.0,-69.0,-2093.0,-878.0,-1398.0
1007,22653.0,8985.0,7078.0,3596.0,975.0,3466.0,3209.0,4903.0,3343.0,637.0,5717.0,112.0,163.0,2379.0,17.0,299.0,1576.0,2725.0,19274.0,57.0,-319.0,467.0,74.0,-475.0,405.0,81.0,181.0,252.0,-37.0,7.0,564.0,65.0,-410.0,182.0,2134.0,153.0,-279.0,-308.0,-847.0
1009,57645.0,23868.0,20934.0,9866.0,2291.0,8567.0,9172.0,13877.0,9729.0,1650.0,6825.0,954.0,215.0,5497.0,344.0,823.0,572.0,6922.0,41857.0,143.0,-505.0,1210.0,41.0,315.0,-1165.0,-8.0,271.0,536.0,-153.0,865.0,546.0,-48.0,-1729.0,254.0,3758.0,-337.0,110.0,-1038.0,-1176.0


In [24]:
# Splitting dataset for predictions
# X = diff_2020_2014.drop('Deaths', axis=1)
X = ml_df
# y = diff_2020_2014.Deaths

In [25]:
# Scaling X learning data
scaler = preprocessing.MinMaxScaler()
names = X.columns
d = scaler.fit_transform(X)
X = pd.DataFrame(d, columns=names)

In [26]:
X.head()

,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_NOHSDP,E_AGE65,E_AGE17,E_DISABL,E_SNGPNT,E_MINRTY,E_LIMENG,E_MUNIT,E_MOBILE,E_CROWD,E_NOVEH,E_GROUPQ,E_UNINSUR,E_DAYPOP,Deaths,diff_E_AGE17,diff_E_AGE65,diff_E_CROWD,diff_E_DAYPOP,diff_E_DISABL,diff_E_GROUPQ,diff_E_HH,diff_E_HU,diff_E_LIMENG,diff_E_MINRTY,diff_E_MOBILE,diff_E_MUNIT,diff_E_NOHSDP,diff_E_NOVEH,diff_E_POV,diff_E_SNGPNT,diff_E_TOTPOP,diff_E_UNEMP,diff_E_UNINSUR
0,0.005316,0.006116,0.005990,0.003797,0.003963,0.003224,0.006096,0.005894,0.008899,0.004745,0.001800,0.000178,0.000925,0.052169,0.001356,0.001830,0.002516,0.002742,0.004180,0.006714,0.692069,0.007124,0.576842,0.096547,0.110649,0.420372,0.031711,0.024770,0.876535,0.038420,0.271194,0.051121,0.939314,0.590011,0.012150,0.996283,0.112301,0.973451,0.996829
1,0.018961,0.030137,0.022265,0.014309,0.013888,0.009541,0.029114,0.018237,0.027808,0.014532,0.004424,0.001841,0.020860,0.144617,0.002554,0.003821,0.014860,0.013371,0.017542,0.029376,0.712167,0.051380,0.593545,0.148273,0.137624,0.438211,0.086353,0.086220,0.871263,0.047066,0.247361,0.058032,0.946512,0.597397,0.021139,0.984238,0.163057,0.963419,0.991809
2,0.002506,0.003054,0.002548,0.003147,0.002680,0.003078,0.003364,0.002336,0.004839,0.003149,0.002008,0.000393,0.000142,0.037636,0.000456,0.001352,0.016329,0.001748,0.002802,0.005165,0.693845,0.004532,0.589660,0.095458,0.096031,0.412185,0.025659,0.018315,0.875425,0.033196,0.331810,0.050582,0.942704,0.593741,0.007540,0.996081,0.107344,0.976218,0.996890
3,0.002058,0.002231,0.001910,0.001908,0.001702,0.002207,0.002519,0.001927,0.003241,0.001814,0.000783,0.000080,0.000179,0.028200,0.000044,0.000489,0.008970,0.001246,0.001752,0.003034,0.695167,0.003943,0.586200,0.098038,0.105764,0.415633,0.025682,0.018488,0.875806,0.035305,0.342148,0.050951,0.941815,0.592382,0.005677,0.997683,0.110808,0.978690,0.997368
4,0.005567,0.006531,0.006185,0.005381,0.004033,0.005563,0.007706,0.005751,0.009986,0.004748,0.000936,0.000683,0.000236,0.065160,0.000880,0.001388,0.003256,0.003279,0.004012,0.008587,0.694401,0.007310,0.585034,0.098982,0.091897,0.407963,0.026187,0.020248,0.875246,0.037257,0.340189,0.049212,0.931791,0.594161,0.008705,0.994147,0.111551,0.975524,0.997083


In [27]:
len(X)

2847

In [28]:
len(y)

2847

# Multifactor Regression Approach

In [29]:
# Splitting X and y into training datasets
kf = KFold(n_splits=10, shuffle=True)
# kf = KFold(n_splits=5,random_state=42,shuffle=True)
for train_index,val_index in kf.split(X):
    X_train,X_test = X.iloc[train_index],X.iloc[val_index],
    y_train,y_test = y.iloc[train_index],y.iloc[val_index]

    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)

    # testing model for general accuracy
    print(f'\nModel score on testing data: {regr.score(X_test, y_test)}')
    print(f'Model score on training data: {regr.score(X_train, y_train)}')

    # coef_graph()


Model score on testing data: 0.24038183467255914
Model score on training data: 0.8838462378965374

Model score on testing data: 0.30436254532434615
Model score on training data: 0.8843630894728591

Model score on testing data: 0.7060063036861124
Model score on training data: 0.8796076961000733

Model score on testing data: 0.7427385653310781
Model score on training data: 0.8763783224364746

Model score on testing data: 0.8901603806185421
Model score on training data: 0.8516734602603746

Model score on testing data: 0.07146712673637712
Model score on training data: 0.8904127836610122

Model score on testing data: 0.7939248969610683
Model score on training data: 0.8795553382721369

Model score on testing data: 0.8912121369665378
Model score on training data: 0.8633985682174878

Model score on testing data: 0.7327259559550272
Model score on training data: 0.8784489229319861

Model score on testing data: 0.6985868642218362
Model score on training data: 0.8825107875039978


# Deep Learning Approach
https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [30]:
# import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow_addons.metrics import RSquare
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasRegressor

In [31]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(19, input_shape=(19,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    # model.compile(loss='mse', optimizer='rmsprop', metrics=[RSquare()])
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mse'])

    return model

In [32]:
estimator = KerasRegressor(model=baseline_model, epochs=50, batch_size=10, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scikeras\wrappers.py", line 762, in fit
    self._fit(
  File "C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scikeras\wrappers.py", line 931, in 

#### Notes

__r2 metric__
- (19, 38, 1) - model.compile(loss='mse', optimizer='rmsprop', metrics=[RSquare()]) - epochs=200, batch_size=10, verbose=0 - 51%
- (19, 10, 1) - model.compile(loss='mse', optimizer='rmsprop', metrics=[RSquare()]) - epochs=200, batch_size=10, verbose=0 - 39%

# Implmenting Automatic Hyperparameter Tuning
https://towardsdatascience.com/hyperparameter-tuning-with-keras-tuner-283474fbfbe

In [ ]:
from keras_tuner import HyperModel
from tensorflow.keras import models, layers

class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape 
   
    def build(self, hp):
        model = Sequential()
        model.add(
            layers.Dense(
                units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                input_shape=input_shape
            )
        )
        
        model.add(
            layers.Dense(
                units=hp.Int('units', 16, 64, 4, default=16),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu')
            )
        )
        
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(1))
        
        model.compile(
            optimizer='rmsprop',loss='mse',metrics=['mse']
            # loss='mse', optimizer='rmsprop', metrics=[RSquare()]
        )
        
        return model

In [ ]:
input_shape = (X_train.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

In [ ]:
from keras_tuner import RandomSearch
tuner_rs = RandomSearch(
            hypermodel,
            objective='mse',
            seed=42,
            max_trials=10,
            executions_per_trial=2)

In [ ]:
tuner_rs.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=0)

In [ ]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
loss, mse = best_model.evaluate(X_test, y_test)